In [ ]:
#using Markdown; Markdown.parse_file("README.md")

In [ ]:
using Plots

In [ ]:
MODEL_PATH_KEY = :wNB;
const FLTYPE = Float32;
include("heavyloadings.jl");
include("iterations.jl")

In [ ]:
half_b = load(MODEL_FOLDER*"\\bunny\\half_bunny_unmod.ply");
half_b_rot = load(MODEL_FOLDER*"\\bunny\\half_bunny_rot1.ply");
dst = load(MODEL_FOLDER*"\\darmstadt\\darmstadt.ply");

In [ ]:
include("firsticp.jl")

In [ ]:
half_b_sc = scaleMesh(half_b,10);
half_b_rot_sc = scaleMesh(half_b_rot,10);

In [ ]:
vis = Visualizer();

In [ ]:
open(vis)
#IJuliaCell(vis)

In [ ]:
placeMeshInLife(vis,half_b_sc,"hbunny","darkred");

In [ ]:
placeMeshInLife(vis,half_b_rot_sc,"rotbunny","olivedrab2")

In [ ]:
sc_d = scaleMesh(dst,0.1);

In [ ]:
placeMeshInLife(vis,sc_d,"darmstadt","blue")

In [ ]:
using CuArrays
function wrapICP2(rfpc,redpc, ftype = Float32; itNum = 10, sampPC = 50, rejPC = 20)
    aref = convert2HomCoordMatrix(vertices(rfpc), ftype)
    caref = cu(aref)
    ared = convert2HomCoordMatrix(vertices(redpc), ftype)
    cared = cu(ared)
    ktr = KDTree(aref, reorder = false)
    retD = iterateEasyPeasy(caref, cared, ktr, itNum, sampPC, rejPC)
    return retD
end

In [ ]:
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);

In [ ]:
a1, a2, kr = initICP(reference_pc, reading_pc);

In [ ]:
resD = wrapICP1(reference_pc, reading_pc, FLTYPE, sampPC = 10, itNum = 30, rejPC = 10);

In [ ]:
resD3 = wrapICP3(reference_pc, reading_pc, FLTYPE, sampPC = 10, itNum = 30);

In [ ]:
timA, tim2A, hmD, errA = postProcDict(resD);
timA3, tim2A3, hmD3, errA3 = postProcDict(resD3)
endhM = sumcumHM(hmD);
endhM3 = sumcumHM(hmD3);
eTR = AffineMap(endhM[1:3,1:3],endhM[1:3,4]);
eTR3 = AffineMap(endhM3[1:3,1:3],endhM3[1:3,4]);

In [ ]:
scatter([tim2A,tim2A3],[errA,errA3])

In [ ]:
resettransform!(vis["rotbunny"])

In [ ]:
settransform!(vis["rotbunny"],eTR)

In [ ]:
settransform!(vis["rotbunny"],eTR3)

In [ ]:
histogram([timA, timA3], bins = 99)

In [ ]:
histogram(timA, xlims = (950,2000), bins = 300)